In [1]:
import re
import json
import os
import sys

from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

import pandas as pd
print(pd.__version__)

parent_path = os.path.abspath('..')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../../')
sys.path.append(parent_path)

from core.ShoreNet.conf import get_data_path

DATA_PATH = get_data_path()

# load .env file
project_path = os.path.abspath('../../')
dotenv_path = os.path.join(project_path, 'secret', '.env')

from dotenv import load_dotenv
load_dotenv(dotenv_path)

# load project variables
from core.ShoreNet.definitions.variables import VariablesManager

var = VariablesManager()
print(var.data_path)


2.1.4
/mnt/d/data/sisi


# 1 Get standard indicator of trust score



# 1.1 choose some docks (top docks for coal events) as standard dock polygon

as the result, choose:  dock_id = [30, 116, 194, 17, 178]


In [2]:
from core.ShoreNet.events.generic.tools import load_events_with_dock
from core.ShoreNet.events.generic.tools import load_dock_polygon
from core.ShoreNet.events.filter import clean_up_events

# -. load dock polygons
dock_polygon_df = load_dock_polygon(var.engine)
coal_dock_polygon_df = dock_polygon_df[dock_polygon_df['type_id'] == 1]

# -. load events with dock
dock_events_df = load_events_with_dock(year=2023, con=var.engine)
coal_dock_events_df = dock_events_df.loc[dock_events_df['coal_dock_id'].isin(coal_dock_polygon_df['dock_id'])]
coal_dock_events_df = clean_up_events(coal_dock_events_df, var)
# coal_dock_events_df = coal_dock_events_df.apply(lambda row: row if ~row['name'].startswith("dbscan") else None, axis=1)
coal_dock_events_df

original event data shape: (248498, 12)
cleaned event data shape: (179925, 12)


,event_id,mmsi,begin_time,end_time,duration,begin_lng,begin_lat,avg_speed,event_categories,coal_dock_id,year,month
9,2023010100002883111541336285010,413362850,1672531213,1672733605,202392,108.357,21.5293,0.000000,low_speed_event,1640,2023,1
15,2023010100002883111641350417010,413504170,1672531200,1672538718,7518,108.333,21.6203,0.000000,low_speed_event,137,2023,1
29,2023010100002885111741235584000,412355840,1672531208,1672548769,17561,108.597,21.7073,0.000000,stop_event_poly,173,2023,1
40,2023010100002886109041352038010,413520380,1672531227,1672634081,102854,108.621,19.1011,0.000000,low_speed_event,172,2023,1
125,2023010100002910111441246403010,412464030,1672531226,1672539577,8351,111.074,21.4774,0.000000,low_speed_event,184,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2940027,2023123121302996129941228258200,412282582,1704058229,1704061434,3205,119.670,39.9274,0.040804,stop_event_poly,10,2023,12
2940122,2023123122233003126010090199020,100901990,1704061424,1704066038,4614,120.280,36.0148,0.000000,record_poly,19,2023,12
2940188,2023123123043003122041383800310,413838003,1704063866,1704065686,1820,120.360,32.0096,0.473650,low_speed_event,199,2023,12
2940190,2023123123063003121941378592510,413785925,1704064007,1704066790,2783,120.336,31.9927,0.642624,low_speed_event,201,2023,12


In [3]:
# -. get dock event count, find top 10 docks
top_ten_coal_dock_series = coal_dock_events_df['coal_dock_id'].value_counts().head(50)
top_ten_coal_dock_df = pd.DataFrame(
    {
        'coal_dock_id': top_ten_coal_dock_series.index,
        'count': top_ten_coal_dock_series.values
    }
)
top_ten_coal_dock_df = pd.merge(
    left=top_ten_coal_dock_df,
    right=coal_dock_polygon_df.loc[:, ['dock_id', 'name']],
    left_on='coal_dock_id',
    right_on='dock_id',
    how='left'
)
# top_ten_coal_dock_df.loc[:, ['dock_id', 'name', 'count']]  # choose: [30, 116, 194, 17, 178]
standard_coal_dock_df = top_ten_coal_dock_df.loc[top_ten_coal_dock_df['dock_id'].isin([30, 116, 194, 17, 178])]
standard_coal_dock_df

,coal_dock_id,count,dock_id,name
3,30,9757,30,舟山港煤炭1号码头 - 北沦港
4,194,9486,194,苏州太仓武港
7,116,5728,116,曹妃甸散货码头1号
14,17,3685,17,青岛码头3号
33,178,1386,178,东莞市广东中远海运重工有限公司2（煤炭）


## 1.2 calculate some indicator value for each chosen dock

there is no relation between mmsi_nunqie, event_count and avg_duration. each dock has its own distribution of these three values.

In [4]:
# -. calculate some indicator value for each chosen dock
indicator_values = []
for _, row in standard_coal_dock_df.iterrows():
    dock_id = row['dock_id']
    dock_name = row['name']
    for month in range(1, 12+1):
        _month_dock_events = coal_dock_events_df.loc[(coal_dock_events_df['coal_dock_id'] == dock_id) & (coal_dock_events_df['month'] == month)]
        _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1).copy()
        _month_dock_events = _month_dock_events.loc[(_month_dock_events['stop_duration'] > 1800) & (_month_dock_events['stop_duration'] < 7 * 24 * 3600)]
        indicator_values.append(
            {
                'dock_id': dock_id,
                'dock_name': dock_name,
                'month': month,
                'mmsi_nunique': _month_dock_events['mmsi'].nunique(),
                'event_count': _month_dock_events.shape[0],
                'avg_duration': _month_dock_events['stop_duration'].mean(),
            }
        )
indicator_values_df = pd.DataFrame(indicator_values)
indicator_values_df

,dock_id,dock_name,month,mmsi_nunique,event_count,avg_duration
0,30,舟山港煤炭1号码头 - 北沦港,1,214,389,33836.107969
1,30,舟山港煤炭1号码头 - 北沦港,2,338,726,39903.561983
2,30,舟山港煤炭1号码头 - 北沦港,3,421,1046,44211.056405
3,30,舟山港煤炭1号码头 - 北沦港,4,157,317,41479.331230
4,30,舟山港煤炭1号码头 - 北沦港,5,416,891,36728.285073
5,30,舟山港煤炭1号码头 - 北沦港,6,529,1151,35847.305821
6,30,舟山港煤炭1号码头 - 北沦港,7,570,1229,40307.994304
7,30,舟山港煤炭1号码头 - 北沦港,8,313,427,28985.690867
8,30,舟山港煤炭1号码头 - 北沦港,9,426,904,37835.378319
9,30,舟山港煤炭1号码头 - 北沦港,10,422,866,43363.129330


# 1.3 calculate density of effectual area for each dock
1. dbscan events for each dock
2. calculate density of effectual area(cluster_id != -1) for each dock
3. get avg density as trust_score. see how it works.

dbscan parameters:
1. events_count(samples_num) = 80
2. radius = 100m

In [12]:
import numpy as np
from sklearn.cluster import DBSCAN

from core.python.utils.plot import plot_scatter
from core.ShoreNet.analyze.polygon.trust_score import trust_score, effectual_event_percentage
from core.ShoreNet.analyze.departure_arrival_docks import calculate_dd_event_count_month

indicator_values = []
month_dock_events_list = []
for _, row in standard_coal_dock_df.iterrows():
    dock_id = row['dock_id']
    dock_name = row['name']
    for month in range(1, 12+1):
        _month_dock_events = coal_dock_events_df.loc[(coal_dock_events_df['coal_dock_id'] == dock_id) & (coal_dock_events_df['month'] == month)]
        _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1)
        _month_dock_events = _month_dock_events.loc[(_month_dock_events['stop_duration'] > 1800) & (_month_dock_events['stop_duration'] < 7 * 24 * 3600)]

        coords = _month_dock_events[[var.column_names.lng_column_name, var.column_names.lat_column_name]].values
        db = DBSCAN(
            eps=0.1 / var.geo_param.kms_per_radian,  # convert 0.1m to radian.
            min_samples=20,
            algorithm='ball_tree',
            metric='haversine'
        ).fit(np.radians(coords))
        _month_dock_events.loc[:, 'cluster_id'] = db.labels_
        # print(f"dock name: {dock_name}, before cluster events shape: {_month_dock_events.shape}")
        before_cluster_events_count = _month_dock_events.shape[0]
        _month_dock_events = _month_dock_events.loc[_month_dock_events['cluster_id'] != -1]
        # print(f"dock name: {dock_name}, after cluster events shape: {_month_dock_events.shape}")
        after_cluster_events_count = _month_dock_events.shape[0]
        indicator_values.append(
            {
                'dock_id': dock_id,
                'dock_name': dock_name,
                'month': month,
                'before_cluster_count': before_cluster_events_count,
                'after_cluster_count': after_cluster_events_count
            }
        )

        # -. plot scatter
        plot_scatter(
            df=_month_dock_events,
            x_col=var.column_names.lng_column_name,
            y_col=var.column_names.lat_column_name,
            output_file_path=os.path.join(var.data_path, var.dp_names.output_path, 'html', f'dbscan_{dock_name}_{month}.html')
        )

        # -. calculate density of each cluster
        _month_dock_events.to_csv(
            os.path.join(var.data_path, var.dp_names.output_path, 'csv', f'dbscan_{dock_name}_{month}.csv'),
            index=False
        )
        month_dock_events_list.append(_month_dock_events)

        # -. calculate trust score (event_count_per_mmsi)
        _month_dock_trust_score = trust_score(_month_dock_events)
        print(f"dock name: {dock_name}, month: {month}, trust score: {_month_dock_trust_score}")

        # -. calculate departure&arrival ship count
        departure_event_count, arrival_event_count= calculate_dd_event_count_month(
            year=2023,
            month=month,
            dock_id=dock_id,
            con=var.engine
        )
        print(f"dock name: {dock_name}, month: {month}, departure count: {departure_event_count}, arrival count: {arrival_event_count}")

        # -. calculate low_speed / stop_event
        event_category_distribute_percentage = _month_dock_events['event_categories'].value_counts(normalize=True)
        percentage_val = effectual_event_percentage(event_category_distribute_percentage)
        print(f"dock name: {dock_name}, month: {month}, effectual event percentage: {percentage_val}%")

        # -. calculate low_speed / stop_event in dbscan cluster
        for cluster_id, group in _month_dock_events.groupby('cluster_id'):
            event_category_distribute_percentage = group['event_categories'].value_counts(normalize=True)
            percentage_val = effectual_event_percentage(event_category_distribute_percentage)
            print(f"cluster_id: {cluster_id}, effectual event percentage: {percentage_val}%")
        print(f">>>>>>>>>>>>>>>>>>>>>>> {dock_name} >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        break

indicator_values_df = pd.DataFrame(indicator_values)
indicator_values_df

dock name: 舟山港煤炭1号码头 - 北沦港, month: 1, trust score: 1.6929297733959223
dock name: 舟山港煤炭1号码头 - 北沦港, month: 1, departure count: 202, arrival count: 124
dock name: 舟山港煤炭1号码头 - 北沦港, month: 1, effective event percentage: 90.25270758122743%
cluster_id: 0, effective event percentage: 100.0%
cluster_id: 1, effective event percentage: 100.0%
cluster_id: 2, effective event percentage: 100.0%
cluster_id: 3, effective event percentage: 100.0%
cluster_id: 4, effective event percentage: 100.0%
cluster_id: 5, effective event percentage: 68.6046511627907%
cluster_id: 6, effective event percentage: 100.0%
>>>>>>>>>>>>>>>>>>>>>>> 舟山港煤炭1号码头 - 北沦港 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
dock name: 苏州太仓武港, month: 1, trust score: 1.8884547659643567
dock name: 苏州太仓武港, month: 1, departure count: 282, arrival count: 190
dock name: 苏州太仓武港, month: 1, effective event percentage: 8.294930875576037%
cluster_id: 0, effective event percentage: 0%
cluster_id: 1, effective event percentage: 14.285714285714285%
cluster_id: 2, e

,dock_id,dock_name,month,before_cluster_count,after_cluster_count
0,30,舟山港煤炭1号码头 - 北沦港,1,389,277
1,194,苏州太仓武港,1,457,217
2,116,曹妃甸散货码头1号,1,384,100
3,17,青岛码头3号,1,92,56
4,178,东莞市广东中远海运重工有限公司2（煤炭）,1,84,20


In [21]:
# -. calculate density of each cluster
df = month_dock_events_list[0]  # dock_id = 30, month = 1, 宁波北沦港
cluster_gdf = df.groupby('cluster_id')

# np.mean([group.shape[0] for cluster_id, group in cluster_gdf])
for cluster_id, group in cluster_gdf:
    print(f"cluster_id: {cluster_id}, density: {group.shape[0] / group['mmsi'].nunique()}")

    mmsi_events_count_list = [mmsi_group.shape[0] for mmsi, mmsi_group in group.groupby('mmsi')]
    print(f"cluster_id: {cluster_id}, avg density: {np.mean(mmsi_events_count_list)}")

cluster_id: 0, density: 1.1818181818181819
cluster_id: 0, avg density: 1.1818181818181819
cluster_id: 1, density: 1.105263157894737
cluster_id: 1, avg density: 1.105263157894737
cluster_id: 2, density: 1.0769230769230769
cluster_id: 2, avg density: 1.0769230769230769
cluster_id: 3, density: 1.375
cluster_id: 3, avg density: 1.375
cluster_id: 4, density: 1.0
cluster_id: 4, avg density: 1.0
